In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [ ]:
weather_codes = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Drizzle: Light intensity",
    53: "Drizzle: Moderate intensity",
    55: "Drizzle: Dense intensity",
    56: "Freezing Drizzle: Light intensity",
    57: "Freezing Drizzle: Dense intensity",
    61: "Rain: Slight intensity",
    63: "Rain: Moderate intensity",
    65: "Rain: Heavy intensity",
    66: "Freezing Rain: Light intensity",
    67: "Freezing Rain: Heavy intensity",
    71: "Snow fall: Slight intensity",
    73: "Snow fall: Moderate intensity",
    75: "Snow fall: Heavy intensity",
    77: "Snow grains",
    80: "Rain showers: Slight intensity",
    81: "Rain showers: Moderate intensity",
    82: "Rain showers: Violent intensity",
    85: "Snow showers: Slight intensity",
    86: "Snow showers: Heavy intensity",
    95: "Thunderstorm: Slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}


In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 14.5869,
	"longitude": 121.0614,
    "city": "Pasig City, Philippines",
	"current": ["temperature_2m", "relative_humidity_2m", "weather_code"],
	"timezone": "Asia/Singapore"
}
weather_data = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = weather_data[0]

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()
current_weather_code = current.Variables(2).Value()

print(f"Current time {pd.to_datetime(current.Time(), unit='s', utc=True)}")
print(f"Current temperature_2m {current_temperature_2m}")
print(f"Current relative_humidity_2m {current_relative_humidity_2m}")
print(f"Current weather_code {current_weather_code}")


In [ ]:
# Initialize an empty dictionary
weather_fc = {}

# Assuming there are 3 variables in order: temperature, humidity, weather code
variable_names = ["temperature", "humidity", "weather_code"]

# Loop through variables and add them to the dictionary
for i, name in enumerate(variable_names):
    weather_fc[name] = current.Variables(i).Value()

# Add time to the dictionary
weather_fc["city"] = params.get("city")
weather_fc["timestamp"] = pd.to_datetime(current.Time(), unit='s')
weather_fc["weather_description"] = weather_codes.get(weather_fc["weather_code"], "Unknown")

# Print the dictionary
print(weather_fc)
